# <center>L2 Computational Physics</center>

## <center>Week 3: Differential Equations I</center>

In [1]:
# usual packages to import
import numpy 
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook, you will generate and plot the decay curve for Iodine-133 analytically and numerically. $^{133}\textrm{I}$ has a half life $t_{1/2}$ of 20.8 hours. This means that half of the nuclei will have decayed after time $t_{1/2}$. Derive the mean lifetime $\tau$ from that information.

In [82]:
# define a function to calculate the mean lifetime from the half life
def meanLifetime(halfLife):
    return halfLife/numpy.log(2)

T_HALF = 20.8
TAU = meanLifetime(T_HALF)


Check your average lifetime:

In [83]:
# this test is worth 1 mark
assert numpy.isclose(TAU, 30.0080568505)         

### The Decay Equation

Implement the function `f_rad` such that the differential equation 

$$ \frac{dN}{dt} = f_{rad}(N,t)$$

describes the radioactive decay process.

- *Your function should return values using hours as the time unit.*
- *The function should use the constant* `TAU`.

In [102]:
def f_rad(N, t):
    return (-1/TAU)*N
    # -Landa*N

Make sure your function works:

In [103]:
# this test cell is worth 1 mark
assert numpy.isclose(f_rad(1000, 0), -33.324383681)           

Solve this first order, ordinary differential equation analytically. Implement this function below, naming it `analytic`. The function should take an initial number of atoms `N0` at time `t=0`, and a time argument. The function should return nuclei count at the time argument. Make sure the function also works for numpy arrays.

In [104]:
def analytic(N0, t):
    return N0*numpy.exp((-1*t)/TAU)

Check your answer for a single time:

In [89]:
# this test is worth 1 mark
assert numpy.isclose(analytic(1000, 41.6), 250.0)           

In [90]:
# this test is worth 1 mark
assert numpy.isclose(analytic(1000, numpy.arange(0, 60, 6)), 
                     [1000.        ,  818.77471839,  670.39203948,  548.90005334,
                       449.4254866 ,  367.97822623,  301.29126855,  246.68967356,
                       201.983268  ,  165.37879338]).all()


## Numerically Solving the ODE

We now wish to solve our differential equation numerically. We shall do this using Euler's and RK4 methods.

### Euler's Method

Create a function which takes as its arguments the initial number of atoms, `n0`, the initial time `t0`, the time step, `dt`, and the number of steps to perform, `n_steps`.  This function should return an array of the number of counts at each time step using Euler's method. This array should contain the initial and final values, so the array length should be `n_steps+1` 

In [141]:
def solve_euler(f, n0, t0, dt, n_panels):
    list = numpy.array([analytic(n0,t0)])
    list = numpy.append(list,list[0]+dt*f(analytic(n0,t0+(0)*dt),t0+(0)*dt))
    for i in range(2,n_panels+1):
        list = numpy.append(list,list[i-1]+dt*f(list[i-1],t0+(i-1)*dt))
    return list



Try your solution:

In [142]:
# this test is worth 1 mark
assert len(solve_euler(f_rad, 1000, 0, 1, 17)) == 18

In [143]:
# this test is worth 2 marks
assert numpy.isclose(solve_euler(f_rad, 1000, 0, 6, 1), [1000.,  800.05369792]).all()

In [144]:
# this test is worth 2 mark
assert numpy.isclose(solve_euler(f_rad, 1000, 0, 6, 10), [1000.        ,  800.05369792,  640.08591955,  512.10310692,
                                                409.7099844 ,  327.7899881 ,  262.24959212,  209.81375595,
                                                167.86227132,  134.29883091,  107.4462763 ]).all()

### RK 4 method

Implement the RK4 method in the `solve_RK4` function. The arguments are the same as for `solve_euler`.

In [158]:
def solve_RK4(f, n0, t0, dt, nsteps):
    listR = numpy.array([analytic(n0,t0)])
    k1 = f(analytic(n0,t0),t0)
    k1jump = analytic(n0,t0) + k1*dt/2        
    k2 = f(k1jump,t0+dt/2)
    k2jump = analytic(n0,t0) + k2*dt/2       
    k3 = f(k2jump,t0+dt/2)
    k3jump = analytic(n0,t0) + k3*dt     
    k4 = f(k3jump,t0+dt/2)
    k=(1/6)*(k1+2*k2+2*k3+k4)
    listR = numpy.append(listR,listR[0]+dt*k)
    for i in range(2,nsteps+1):
        k1 = f(listR[i-1],t0+i*dt)
        k1jump = listR[i-1] + k1*dt/2        
        k2 = f(k1jump,t0)
        k2jump = listR[i-1] + k2*dt/2       
        k3 = f(k2jump,t0)
        k3jump = listR[i-1] + k3*dt     
        k4 = f(k3jump,t0)
        k=(1/6)*(k1+2*k2+2*k3+k4)
        
        
        listR = numpy.append(listR,listR[i-1]+dt*k)
    return listR


In [159]:
# This checks that we return an array of the right length
# this test is worth 1 mark
assert len(solve_RK4(f_rad, 1000, 0, 1, 17)) == 18

In [160]:
# This checks that a single step is working
# this test is worth 2 mark
assert numpy.isclose(solve_RK4(f_rad, 1000,0, 6, 1), [1000.,  818.7773]).all()

In [161]:
# This checks multiple steps
# this test is worth 2 marks
assert numpy.isclose(solve_RK4(f_rad, 1000, 0, 6, 10), [
    1000.,
    818.77729521,  
    670.39625915,  
    548.90523578,
    449.43114428,  
    367.9840167,  
    301.29695787,  
    246.69510822, 
    201.98835345,  
    165.3834777,  
    135.41223655]).all()

## Plotting task

**Task 1: **

Create a plot to show that the RK4 method has an error that scales better with the number of steps than the Euler method. (click on the "+" button to create new cells.)       [task worth 5 marks]


In [175]:
dt = numpy.array([])
DiffRK4Ana = ([])
DiffEulAna = ([])
nsteps = numpy.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
dt = 10/nsteps
AnalyticMethod = analytic(1000,10)
#for i in range(0,8):
    #nstep = nsteps[i]
DiffRK4Ana = numpy.append(DiffRK4Ana,solve_RK4(f_rad, 1000, 0, 1, nstep)[nsteps] - AnalyticMethod)
    #DiffRK4Ana = solve_RK4(f_rad, 1000, 0, 1, nstep) - AnalyticMethod
print(DiffRK4Ana)



#DiffEulAna = solve_euler(f_rad, 1000, 0, 6, nsteps)[nsteps+1] - AnalyticMethod


#y = []
#def updated(x): return (result_analytic-integrate_numeric(x0, x1, x))/result_analytic
#y = list(map(updated, panel_counts))


[ 2.83404559e+02  2.50629317e+02  2.18928290e+02  1.88266273e+02
  1.58609210e+02  1.29924165e+02  1.02179280e+02  7.53437393e+01
  4.93877403e+01  2.42824555e+01  2.52329892e-06 -2.34865872e+01
 -4.62033982e+01 -6.81756602e+01 -8.94277760e+01 -1.09983349e+02
 -1.29865207e+02 -1.49095433e+02 -1.67695384e+02 -1.85685716e+02
 -2.03086412e+02 -2.19916795e+02 -2.36195558e+02 -2.51940781e+02
 -2.67169950e+02 -2.81899980e+02 -2.96147229e+02 -3.09927521e+02
 -3.23256161e+02 -3.36147952e+02 -3.48617210e+02 -3.60677786e+02
 -3.72343074e+02 -3.83626029e+02 -3.94539182e+02 -4.05094654e+02
 -4.15304168e+02 -4.25179063e+02 -4.34730305e+02 -4.43968504e+02
 -4.52903918e+02 -4.61546471e+02 -4.69905763e+02 -4.77991077e+02
 -4.85811393e+02 -4.93375397e+02 -5.00691488e+02 -5.07767792e+02
 -5.14612169e+02 -5.21232220e+02 -5.27635297e+02 -5.33828511e+02
 -5.39818742e+02 -5.45612641e+02 -5.51216644e+02 -5.56636974e+02
 -5.61879652e+02 -5.66950499e+02 -5.71855149e+02 -5.76599047e+02
 -5.81187463e+02 -5.85625